In [ ]:
import pickle
import json
import networkx as nx
import datetime
import pandas
import pandas as pd 
from tqdm import tqdm
import numpy as np

 # Identifying HF patients 

In [ ]:
def graph_from_onto(onto):
    G = nx.DiGraph()
    cl_edges = []
    for s, ts in onto.items():
        for t in ts:
            cl_edges.append((s.replace('S-', ''), t.replace('S-', '')))
    G.add_edges_from(cl_edges)
    return G

def load_onto(o):
    with open(f'/projects/data/GS/{o}', 'rb') as f:
        onto = pickle.load(f)
    G = graph_from_onto(onto)
    return G

def expand_codes(onto, codes):
    expanded = {}
    for name, top_codes in codes.items():
        expanded[name] = set()
        for c in top_codes:
            expanded[name].add(c)
            if c in onto:
                expanded[name].update(nx.ancestors(onto, c))
            else:
                print("NOT FOUND:", c, name)
        print(name, len(top_codes), len(expanded[name]))
    return expanded

In [ ]:
onto = load_onto('isa_rela_ch2pt_202009.pickle')

In [ ]:
hf = { 'HF' : ['84114007']}
hf = expand_codes(onto,hf)

In [ ]:
hf_codes = set()
for x in hf.values():
    hf_codes.update(x)

In [ ]:
%%time 
with open('/projects/data/GS/pt2cui_pos_dates.pickle', 'rb') as f:
    pt2cui_pos_dates = pickle.load(f)

In [ ]:
# identifying patients with at least 1 mention of HF 

def build_set(codes_list):
    '''
    this function returns 
    :param codes:
    :return data: 
    '''
    data = set()
    for pt in pt2cui_pos_dates:
        for ptcodes in pt2cui_pos_dates[pt]:
            if ptcodes in codes_list:
                data.add(pt) 
    return data

In [ ]:
hf_patients = build_set(hf_codes)

In [ ]:
len(hf_patients)

In [ ]:
hf_patients

In [ ]:
pt2cui_pos_dates = {k: pt2cui_pos_dates[k] for k in hf_patients} 

In [ ]:
hf_ids = list(hf_patients)
hf_ids = pandas.DataFrame(data={"hf_ids": hf_ids})
hf_ids.to_csv('/projects/RALES TRIAL/1A/spiro dataextraction/spiro data/hf_ids.csv', sep=',',index=False)

#  HF patients on spironolactone (structured data)

In [ ]:
df = pd.read_csv('/projects/data/GS/HF/gs_all_orders.csv') # load the orders table
df = df[['client_idcode', 'order_name', 'order_summaryline', 'order_entered', 'order_arrivaldtm']]
hf = df['client_idcode'].isin(hf_patients) # identify HF patients 
df = df[hf] # only keep HF patients 

In [ ]:
df['drug_name'] = df['order_name'].str.upper()

In [ ]:
%%time
drug_names = ['SPIRONOLACTONE']
keep = []
for index, row in df.iterrows():
    keep.append(any([x in row['drug_name'] for x  in drug_names]))

In [ ]:
df = df.loc[keep] # only keeping patients on spironolactone

In [ ]:
# only keeping the first spironolactone prescription order line

df['order_arrivaldtm'] = pd.to_datetime(df['order_arrivaldtm'], utc=True).dt.date 
min_date = df.groupby('client_idcode').order_arrivaldtm.min()
df = df.merge(min_date, on='client_idcode',suffixes=('', '_min'))
df = df[df.order_arrivaldtm==df.order_arrivaldtm_min].drop('order_arrivaldtm_min', axis=1)

In [ ]:
df['order_summaryline_b'] = df['order_summaryline'].str.upper()

In [ ]:
# removing patients with a mention of those words below

matches = ['TWICE','HELD', 'ON HOLD', 'STOPPED', 'STOP', 'WITHHELD', 'WITHHOLD', 'INCREASED', 'RESTARTED', 'INCREASE',
           'DECREASED', 'DECREASE','BD', 'B.D.', 'TDS', 'T.D.S.', 'DISCONTINUED', 'DISCONTINUE']
ids_to_exclude = set()

for i,row in tqdm(df.iterrows()):
    if any(x in df.loc[i,'order_summaryline_b'] for x in matches):
        ids_to_exclude.add(df.loc[i,'client_idcode']) 

In [ ]:
# code interruption: double checking individuals in icu 

matches = ['(PRESCRIBED IN ICU)']
icu_double_check = set()

for i,row in tqdm(df.iterrows()):
    if any(x in df.loc[i,'order_summaryline_b'] for x in matches):
        icu_double_check.add(df.loc[i,'client_idcode']) 
len(icu_double_check)

In [ ]:
with open ('/projects/RALES TRIAL/1A/spiro dataextraction/spiro data/icu_double_check.pickle', 'wb') as f:
    pickle.dump(icu_double_check,f)

In [ ]:
len(ids_to_exclude)

In [ ]:
df = df[~df.client_idcode.isin(ids_to_exclude)]

In [ ]:
# extracting the dose of the prescription order

for i,row in tqdm(df.iterrows()):
    if '(PRESCRIBED IN ICU)' not in df.loc[i,'order_summaryline_b']:
        df.loc[i,'dose'] = df.loc[i,'order_summaryline_b'].split('MG')[0].strip()[0:4].strip()
    else:
        df.loc[i,'dose'] = np.nan

for i,row in tqdm(df.iterrows()):
    if '(PRESCRIBED IN ICU)' in df.loc[i,'order_summaryline_b']:
        if '12.5' not in df.loc[i,'order_summaryline_b']: 
            df.loc[i,'dose'] = df.loc[i,'order_summaryline_b'].split('(PRESCRIBED IN ICU)')[1].strip()[0:2].strip()
        else:
            df.loc[i,'dose'] = df.loc[i,'order_summaryline_b'].split('(PRESCRIBED IN ICU)')[1].strip()[0:4].strip()

In [ ]:
df.loc[df['order_summaryline_b'].str.contains('(PRESCRIBED IN ICU)', case=False)]

In [ ]:
print(df['dose'].isna().sum())
df['dose'] = pd.to_numeric(df['dose'], downcast='float')
df['dose'].unique()

In [ ]:
mask = df.client_idcode.duplicated(keep=False)
df[mask]

In [ ]:
df.loc[df['dose'] == 300]

In [ ]:
result = df.groupby('client_idcode')['dose'].apply(lambda x: x.nunique() == 1).reset_index(name='same_dose_value')
false_rows = result.loc[result['same_dose_value'] == False]
print(false_rows)

In [ ]:
# orders seem to be entered twice for some patients (see cell above), only keeping one row for each patients

df = df.loc[df.groupby('client_idcode')['dose'].idxmin()]
df.reset_index(drop=True, inplace=True)

In [ ]:
print(df.shape)
duplicates = df['client_idcode'].duplicated()
duplicates.value_counts()

#  HF patients on spironolactone (unstructured data)

In [ ]:
# entering the SNOMED codes for spironolactone

spiro = {
    'Spironolactone' : ['387078006','13929005','777603002', '318056008', '1301111000001106', '1261411000001109']
}

In [ ]:
spiro = expand_codes(onto,spiro)

In [ ]:
spiro_codes = set()
for x in spiro.values():
    spiro_codes.update(x)

In [ ]:
# function to identify HF patients with 2 mentions of spironolactone

def first_date_for_code_list(pt_data, codes):
    first = None
    for x in codes:
        if x in pt_data and len(pt_data[x])>=2:  
            if first == None:
                first = min(pt_data[x])
            else:
                first = min(first, min(pt_data[x]))
    return first

first_mention_spiro = {}
for pt in hf_patients:
    pt_data = pt2cui_pos_dates.get(pt, {})
    first = first_date_for_code_list(pt_data, spiro_codes) #
    first_mention_spiro[pt] = first

In [ ]:
# filter out the None values (aka non initiators)
    
filtered_first_mention_spiro = {k: v for k, v in first_mention_spiro.items() if v is not None}

In [ ]:
filtered_first_mention_spiro = {k: v for k,v in filtered_first_mention_spiro.items() if k not in ids_to_exclude}

In [ ]:
df_unstructured_data = pd.DataFrame(filtered_first_mention_spiro.keys(), columns = ['client_idcode'] )
df_unstructured_data['spiro_prescription'] = df_unstructured_data['client_idcode'].map(filtered_first_mention_spiro)
df_unstructured_data['spiro_prescription'] = pd.to_datetime(df_unstructured_data['spiro_prescription'], utc=True).dt.date

In [ ]:
df_unstructured_data.head()

In [ ]:
df_unstructured_data.shape

# Comparing the structured and unstructured data

In [ ]:
# keeping patients with no major difference between stuctured/unstrctured spiro presctiptions: if spiro was mentioned in clinical text 91 days before an order being placed patients are removed


def comparison(df1,df2):
    '''
    this function returns 
    :param codes:
    :return data: 
    '''
    df = pd.merge(df1, df2, on='client_idcode', how='outer') 
    df['difference'] =  (pd.to_datetime(df.order_arrivaldtm) - pd.to_datetime(df.spiro_prescription)).dt.days
    df = df.drop(df[df.difference >= 91].index) 
    df.reset_index(drop=True, inplace=True) 
    # del df ['difference']
    return df

In [ ]:
df = comparison(df,df_unstructured_data)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# removing patients for which we don't know the spironolactone dose & dosage

df = df.dropna(subset=['order_summaryline_b'])
df.shape

In [ ]:
df = df[['client_idcode', 'order_arrivaldtm', 'dose']]

In [ ]:
df.head()

In [ ]:
first_mention_spiro = dict(zip(df.client_idcode, df.order_arrivaldtm)) 
len(first_mention_spiro)

In [ ]:
spiro_dose = dict(zip(df.client_idcode, df.dose)) 
len(spiro_dose)

###  First mention spiro date and dose dictionaries

In [ ]:
with open ('/projects/RALES TRIAL/1A/spiro dataextraction/spiro data/first_mention_spiro.pickle', 'wb') as f:
    pickle.dump(first_mention_spiro,f)

In [ ]:
with open ('/projects/RALES TRIAL/1A/spiro dataextraction/spiro data/spiro_dose.pickle', 'wb') as f:
    pickle.dump(spiro_dose,f)

# Checking if the HF diagnosis is before the spironolactone prescription

In [ ]:
# identifying patients with at least 1 mention of HF and getting the date of the first HF mention

def first_date_for_code_list(pt_data, codes):
    first = None
    for x in codes:
        if x in pt_data:  
            if first == None:
                first = min(pt_data[x])
            else:
                first = min(first, min(pt_data[x]))
    return first

first_mention_hf = {}
for pt in hf_patients:
    pt_data = pt2cui_pos_dates.get(pt, {})
    first = first_date_for_code_list(pt_data, hf_codes)
    first_mention_hf[pt] = first

In [ ]:
len(first_mention_hf)

In [ ]:
# from HF dictionary to HF dataframe

hf = pd.DataFrame(first_mention_hf.items(), columns = ['client_idcode', 'first_mention_hf'] )
hf['first_mention_hf'] = pd.to_datetime(hf['first_mention_hf'], utc=True).dt.date

In [ ]:
# merging the spironolactone and hf dataframes

df = pd.merge(df, hf, on='client_idcode', how='left') 

In [ ]:
df['difference'] =  df['order_arrivaldtm'] - df['first_mention_hf']

In [ ]:
# indetifying patients on spironolactone with a mention of HF within 6 months prior the arrival of the spironolactone prescription order

min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = df['difference'] >= min_time 
t2 = df['difference'] <= max_time
in_window = t1 & t2 
df['in_window'] = in_window

In [ ]:
df.head()

In [ ]:
df = df[df.in_window]
len(df)

In [ ]:
df.head()

In [ ]:
hf_diagnosis_match = set(df['client_idcode'])
len(hf_diagnosis_match)

In [ ]:
with open ('/projects/RALES TRIAL/1A/spiro dataextraction/spiro data/hf_diagnosis_match.pickle', 'wb') as f:
    pickle.dump(hf_diagnosis_match,f)